In [1]:
import numpy as np
import pandas as pd

from datetime import datetime as dt
import random
import re

from tqdm import tqdm

from itertools import chain
from collections import Counter

In [2]:
data = pd.read_parquet("../../Data/split_4.parquet")

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22053 entries, 66159 to 88211
Data columns (total 24 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   OpportunityId                    22053 non-null  object 
 1   ApplicationId                    22053 non-null  object 
 2   ExternalBriefDescription         22053 non-null  object 
 3   ExternalDescription              22053 non-null  object 
 4   Title                            22053 non-null  object 
 5   JobCategoryName                  22053 non-null  object 
 6   IsRejected                       22053 non-null  bool   
 7   IsCandidateInternal              22053 non-null  bool   
 8   BehaviorCriteria                 22053 non-null  object 
 9   MotivationCriteria               22053 non-null  object 
 10  EducationCriteria                22053 non-null  object 
 11  LicenseAndCertificationCriteria  22053 non-null  object 
 12  SkillCriteria 

In [4]:
data.sample(6)

,OpportunityId,ApplicationId,ExternalBriefDescription,ExternalDescription,Title,JobCategoryName,IsRejected,IsCandidateInternal,BehaviorCriteria,MotivationCriteria,...,Educations,LicenseAndCertifications,Skills,Motivations,Behaviors,StepId,StepName,Tag,StepGroup,pass_first_step
86067,xMoyisvopkqLgt07geT6Jw==,YgGf9RgbdUS+k+4+yLKlaA==,$16.00 PER HOUR\n\nPAID TRAINING!\n\nESSENTIAL...,<p><strong>$16.00 PER HOUR</strong></p>\n<p><s...,Service Technician (Paid Training),Outside Service Technician,False,False,[{'Description': 'Capable of carrying out a gi...,[{'Description': 'Inspired to perform well by ...,...,"[{'Degree': 'High School Diploma', 'Descriptio...",[],"[{'ScaleValue': '2', 'ScaleValueName': 'Some K...",[{'Description': 'Inspired to perform well by ...,[{'Description': 'Works well as a member of a ...,8q2KIlv/tUeV4GfMrbY5Ig==,Recruiter Reviewed,3.0,other,True
77489,ag+P0sM0YUyQ7qxyySDh9Q==,RR9SJGvSHUWbG0XaVGCkhQ==,"$15.00 PER HOUR\n\nAt Orkin, our purpose is to...",<p><strong>$15.00 PER HOUR</strong></p>\n<p><s...,Service Technician (Will Train),Outside Service Technician,True,False,[{'Description': 'Shows firm and constant supp...,[{'Description': 'Inspired to perform without ...,...,"[{'Degree': 'Diploma', 'Description': None, 'G...","[{'Active': False, 'Awarded': None, 'LicenseAn...","[{'ScaleValue': '5', 'ScaleValueName': 'Expert...",[{'Description': 'Inspired to perform well by ...,[{'Description': 'Works well as a member of a ...,K8yQlic+/UiXxBMpOnAoLQ==,Decline,2.0,declined,False
72850,sPHWrqxHD06uXRtd8gGPuw==,GzORvgEXhUauF+VoKEbP3w==,"FIRST-YEAR EARNINGS OPPORTUNITY of $40,000 - $...",<p><strong>FIRST-YEAR EARNINGS OPPORTUNITY of ...,Service Technician (Will Train),Outside Service Technician,False,False,[{'Description': 'Capable of carrying out a gi...,[{'Description': 'Inspired to perform well by ...,...,[],[],[],[],[],DmO2vH8l002ogt2z2t8Z9Q==,In Review,3.0,interview,True
69168,w10viWZORky6B4ACHSRQwQ==,9FgXE9iKOES+mUAxQ8jQng==,We are the pest management industry leader. W...,<p>&nbsp;</p>\n<p>&nbsp;</p>\n<p><strong><img ...,TEST - DO NOT APPLY - Service Technician Train...,Outside Service Technician,True,False,[{'Description': 'Capable of carrying out a gi...,[{'Description': 'Inspired to perform well by ...,...,"[{'Degree': 'BS', 'Description': None, 'Gradua...",[],"[{'ScaleValue': '3', 'ScaleValueName': 'Interm...",[{'Description': 'Inspired to perform well by ...,[{'Description': 'Devoted to a task or purpose...,LBb1wQ0XZUGjXIEEsxTVPQ==,In Review,3.0,interview,True
81351,SA6jPq6PtE20qgB2Iot2mw==,yBklmf+3x0+ZQ36bt9GKrw==,"$16.00 Per Hour\n\nAt Orkin, our purpose is to...",<p><strong>$16.00 Per Hour</strong></p>\n<p><s...,"Service Technician, Termite (Will Train)",Outside Service Technician,True,False,[{'Description': 'Capable of carrying out a gi...,[{'Description': 'Inspired to perform well by ...,...,"[{'Degree': 'GED', 'Description': None, 'Gradu...",[],"[{'ScaleValue': '4', 'ScaleValueName': 'Advanc...",[],[],K8yQlic+/UiXxBMpOnAoLQ==,Decline,2.0,declined,False
76032,uElJb0GGF0m9//AQKHLpIQ==,vHy13A5l6EqcCE/wTboapQ==,"At Orkin, our purpose is to help protect the w...","<h4><strong>At Orkin, our purpose is to help p...",Pest Control Technician,Outside Service Technician,True,False,[{'Description': 'Capable of carrying out a gi...,[{'Description': 'Inspired to perform well by ...,...,"[{'Degree': 'High school diploma', 'Descriptio...",[],"[{'ScaleValue': '5', 'ScaleValueName': 'Expert...",[],[],K8yQlic+/UiXxBMpOnAoLQ==,Decline,2.0,declined,False


In [5]:
data.StepName.value_counts()

StepName
Decline                                10131
In Review                               6605
Phone Interview                         1627
Not Selected                            1144
Reviewed - Not Interested                727
In Person Interview1                     629
Background Check                         388
Recruiter Reviewed                       367
Hire                                     200
In Person Interview                       33
Phone Screen / Interview                  31
In Person Interview 2                     30
Background Check (Criminal & MVR)         27
Background Ck - Crim & MVR                26
Contingent Offer                          23
1st Interview                             22
Reviewed - Considered for other job       15
Reference Interview                        9
Post Offer / Physical & Drug Screen        5
Post-Offer Phys and/or Drug Screen         5
In Person Interview2                       3
In Person Interview #1                     2
E

In [6]:
past_review_steps = [
        "Phone Interview",
        "In Person Interview1",
        "Background Check",
        "Recruiter Reviewed",
        "Hire",
        "In Person Interview",
        "Phone Screen / Interview",
        "In Person Interview 2",
        "Background Check (Criminal & MVR)",
        "Background Ck - Crim & MVR",
        "Contingent Offer",
        "1st Interview",
        "Reference Interview",
        "Post Offer / Physical & Drug Screen",
        "Post-Offer Phys and/or Drug Screen",
        "In Person Interview2",
        "In Person Interview #1",
        "Extend Contingent Offer",
        "In Person Interview #2"
    ]
def categorize_step(step_name, past_review_steps=past_review_steps):
    # List of steps that signify the candidate made it past the resume stage
    return 1 if step_name in past_review_steps else 0

# Apply the function to your DataFrame
data['Step_Category'] = data['StepName'].apply(categorize_step)


In [7]:
data['Step_Category'].value_counts()

Step_Category
0    18622
1     3431
Name: count, dtype: int64

### We categorized the steps since we are interested in who is 'qualified' for the job, since performance on interviews (ultimately what leads the candidate to the hire) is not something we can predict from the resume.
- furthermore, this provides us more data than just looking at hires.

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22053 entries, 66159 to 88211
Data columns (total 25 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   OpportunityId                    22053 non-null  object 
 1   ApplicationId                    22053 non-null  object 
 2   ExternalBriefDescription         22053 non-null  object 
 3   ExternalDescription              22053 non-null  object 
 4   Title                            22053 non-null  object 
 5   JobCategoryName                  22053 non-null  object 
 6   IsRejected                       22053 non-null  bool   
 7   IsCandidateInternal              22053 non-null  bool   
 8   BehaviorCriteria                 22053 non-null  object 
 9   MotivationCriteria               22053 non-null  object 
 10  EducationCriteria                22053 non-null  object 
 11  LicenseAndCertificationCriteria  22053 non-null  object 
 12  SkillCriteria 

In [9]:
data.dtypes

OpportunityId                       object
ApplicationId                       object
ExternalBriefDescription            object
ExternalDescription                 object
Title                               object
JobCategoryName                     object
IsRejected                            bool
IsCandidateInternal                   bool
BehaviorCriteria                    object
MotivationCriteria                  object
EducationCriteria                   object
LicenseAndCertificationCriteria     object
SkillCriteria                       object
WorkExperiences                     object
Educations                          object
LicenseAndCertifications            object
Skills                              object
Motivations                         object
Behaviors                           object
StepId                              object
StepName                            object
Tag                                float64
StepGroup                           object
pass_first_

---

### Fields related to Opportunities
- OpportunityId
- ExternalBriefDescription
- ExternalDescription
- Title
- JobCategoryName

### Fields related to Applications
- ApplicationId
- IsRejected
- IsCandidateInternal
- pass_first_step
- Tag
- WorkExperiences
- Educations
- LicenseAndCertifications
- Skills
- Motivations
- Behaviors
- StepId
- StepName
- StepGroup

### Fields related to Criteria
- BehaviorCriteria
- MotivationCriteria
- EducationCriteria
- LicenseAndCertificationCriteria
- SkillCriteria

In [10]:
criteria_columns = ["BehaviorCriteria", "MotivationCriteria",
    "EducationCriteria", "LicenseAndCertificationCriteria",
    "SkillCriteria",]

for column in criteria_columns:
    print(data[column][66159]) # Random sample

data.drop(columns=criteria_columns, inplace=True)

[{'Description': 'Capable of carrying out a given task with all details necessary to get the task done well', 'Name': 'Detail Oriented', 'Required': 'true'}
 {'Description': 'Devoted to a task or purpose with loyalty or integrity', 'Name': 'Dedicated', 'Required': 'true'}
 {'Description': 'Shows firm and constant support to a cause', 'Name': 'Loyal', 'Required': 'false'}
 {'Description': 'Works well as a member of a group', 'Name': 'Team Player', 'Required': 'false'}]
[{'Description': 'Inspired to perform well by the knowledge that your job is safe', 'Name': 'Job Security', 'Required': 'false'}
 {'Description': 'Inspired to perform well by the completion of tasks', 'Name': 'Goal Completion', 'Required': 'true'}
 {'Description': 'Inspired to perform without outside help', 'Name': 'Self-Starter', 'Required': 'true'}]
[{'Degree': 'High School/GED', 'Major': None, 'Related': 'true', 'Required': 'true'}]
[{'LicenseAndCertification': 'Driver License', 'LicenseAndCertificationId': None, 'Requ

In [11]:
data.columns

Index(['OpportunityId', 'ApplicationId', 'ExternalBriefDescription',
       'ExternalDescription', 'Title', 'JobCategoryName', 'IsRejected',
       'IsCandidateInternal', 'WorkExperiences', 'Educations',
       'LicenseAndCertifications', 'Skills', 'Motivations', 'Behaviors',
       'StepId', 'StepName', 'Tag', 'StepGroup', 'pass_first_step',
       'Step_Category'],
      dtype='object')

### The criteria could increase accuracy in the model, but it's not practical yet given the inputs we expect from a user. We'll drop criteria columns for now.

In [12]:
applicant_columns = ['ApplicationId', 'IsRejected','IsCandidateInternal',
                    'pass_first_step', 'Tag', 'WorkExperiences',
                    'Educations', 'LicenseAndCertifications', 'Skills',
                    'Motivations', 'Behaviors', 'StepId', 'StepName',
                    'StepGroup']

opportunity_columns = ["OpportunityId",
    "ExternalBriefDescription",
    "ExternalDescription",
    "Title",
    "JobCategoryName"]

In [13]:
data['WorkExperiences'][85974]

array([{'EndMonth': 9.0, 'EndYear': 2020.0, 'JobTitle': 'Independent living coach ', 'Location': None, 'Organization': 'Trinity Services ', 'StartMonth': 8.0, 'StartYear': 2016, 'WhatYouDid': 'Assist with daily living ', '_id': 'nAn/awzH7ku7MQo3IC92Ug=='}],
      dtype=object)

### Extracting the job titles and work experiences from applications

In [14]:
data['Skills'][66159]

array([{'ScaleValue': '5', 'ScaleValueName': 'Expert', 'Skill': 'Sales'},
       {'ScaleValue': '5', 'ScaleValueName': 'Expert', 'Skill': 'Cash'},
       {'ScaleValue': '5', 'ScaleValueName': 'Expert', 'Skill': 'Customer service'},
       {'ScaleValue': '5', 'ScaleValueName': 'Expert', 'Skill': 'Marketing'},
       {'ScaleValue': '5', 'ScaleValueName': 'Expert', 'Skill': 'Receptionist'},
       {'ScaleValue': '5', 'ScaleValueName': 'Expert', 'Skill': 'Retail sales'},
       {'ScaleValue': '4', 'ScaleValueName': 'Advanced', 'Skill': 'Payroll'},
       {'ScaleValue': '4', 'ScaleValueName': 'Advanced', 'Skill': 'Inventory'},
       {'ScaleValue': '4', 'ScaleValueName': 'Advanced', 'Skill': 'Point of sale'},
       {'ScaleValue': '4', 'ScaleValueName': 'Advanced', 'Skill': 'Pos'},
       {'ScaleValue': '4', 'ScaleValueName': 'Advanced', 'Skill': 'Pos system'},
       {'ScaleValue': '5', 'ScaleValueName': 'Expert', 'Skill': 'Staffing'},
       {'ScaleValue': '5', 'ScaleValueName': 'Expert',

In [15]:
def extract_details(entry):
    job_titles, job_responsibilities, applicant_educations, skills = [], [], [], []
    
    # Extract Job Details
    work_experience = entry.get('WorkExperiences', [])
    for job in work_experience:
        title = job.get('JobTitle', '')
        responsibility = job.get('WhatYouDid', '')
        if title: job_titles.append(title)
        if responsibility: job_responsibilities.append(responsibility)
    
    # Extract Education Details
    education_list = entry.get('Educations', [])
    for edu in education_list:
        degree = edu.get('Degree', '')
        if degree: applicant_educations.append(degree)
    
    # Extract Skills
    skill_list = entry.get('Skills', [])
    for skill in skill_list:
        skill_name = skill.get('Skill', '')
        if skill_name: skills.append(skill_name)

    return "++".join(job_titles), "++".join(job_responsibilities), "++".join(applicant_educations), "++".join(skills)

# Apply the function and create new columns
data[['Applicant_Job_Titles', 'Applicant_Job_Responsibilities', 'Applicant_Education', 'Applicant_Reported_Skills']] = data.apply(lambda x: pd.Series(extract_details(x)), axis=1)

In [16]:
data[['Applicant_Job_Titles', 'Applicant_Job_Responsibilities', 'Applicant_Education', 'Applicant_Reported_Skills']]

,Applicant_Job_Titles,Applicant_Job_Responsibilities,Applicant_Education,Applicant_Reported_Skills
66159,General Manager++Assistant Manager++General Ma...,30907\nInterviewed job candidates and made sta...,High School Diploma++BPA,Sales++Cash++Customer service++Marketing++Rece...
66160,Package handler,It waa a great job just got home to late in th...,,
66161,Logistics Supervisor++Logistics Warehouse Supe...,* Supervised operations in multi-shift food gr...,Bachelor of Science++Master of Arts,Metrics++Inventory++Training++Operations++Hr++...
66162,Service tech,Diagonose and repair vehicles. Suspension work...,,
66163,"security supervisor ++Sharp Shooter, Navigator...","Patrolling, protecting people, property, infor...",G.E.D,Cpr++Cpr certified++Documenting++Radio communi...
...,...,...,...,...
88207,Delivery Driver,Delivered and sold furniture,,
88208,Cook,"Cooking food, keeping hot box stocked, cleanin...",Diploma,Driving
88209,Auditor,,Diploma,Confined space training
88210,Machine Operator++Thrower++Customer Service Re...,Simple machine operations++On the back of the ...,Diploma,Customer service++Customer service representat...


### Since we will be cleaning up punctuation marks, I separated values using "++", which is unlikely to be used in the data. Therefore, we can still separate the values later on.

In [17]:
data.columns

Index(['OpportunityId', 'ApplicationId', 'ExternalBriefDescription',
       'ExternalDescription', 'Title', 'JobCategoryName', 'IsRejected',
       'IsCandidateInternal', 'WorkExperiences', 'Educations',
       'LicenseAndCertifications', 'Skills', 'Motivations', 'Behaviors',
       'StepId', 'StepName', 'Tag', 'StepGroup', 'pass_first_step',
       'Step_Category', 'Applicant_Job_Titles',
       'Applicant_Job_Responsibilities', 'Applicant_Education',
       'Applicant_Reported_Skills'],
      dtype='object')

In [18]:
data.sample(3)

,OpportunityId,ApplicationId,ExternalBriefDescription,ExternalDescription,Title,JobCategoryName,IsRejected,IsCandidateInternal,WorkExperiences,Educations,...,StepId,StepName,Tag,StepGroup,pass_first_step,Step_Category,Applicant_Job_Titles,Applicant_Job_Responsibilities,Applicant_Education,Applicant_Reported_Skills
74922,N4W2TknxGkqgeEsXJ1sEoQ==,/nn4bd/STkmdhoYtBayBvA==,"FIRST-YEAR EARNINGS OPPORTUNITY of $40,000 - $...",<p><strong>FIRST-YEAR EARNINGS OPPORTUNITY of ...,Service Technician (Will Train),Outside Service Technician,True,False,"[{'EndMonth': 7.0, 'EndYear': 2019.0, 'JobTitl...","[{'Degree': 'Some college', 'Description': Non...",...,DmO2vH8l002ogt2z2t8Z9Q==,In Review,3.0,interview,True,0,Peer Support Mentor & Dog Trainer++Seasonal La...,,Some college++Diploma,Quality control++Tle++Clerk++Mentor++Waiter++T...
85684,bolnXokT9U+2NZhqzDeZYA==,DVFSBboiMEaegaj2RfDlSQ==,"First Year Earnings Opportunity of $65,000 - $...",<p><strong>First Year Earnings Opportunity of ...,Outside Sales Representative,Sales,True,False,"[{'EndMonth': 6.0, 'EndYear': 2019.0, 'JobTitl...",[{'Degree': 'Bachelor of Science in Business A...,...,K8yQlic+/UiXxBMpOnAoLQ==,Decline,2.0,declined,False,0,Store Manager++WMS Project Manager++Office/HR/...,"• Set expectations and provide leadership, coa...",Bachelor of Science in Business Administration,Ada++Equal employment opportunity commission++...
86208,yXXG85d1GUCVvm4nEmPWNg==,MIJf395MXkWl2bK9SUK61g==,"$14.00 PER HOUR\n\nAt Orkin, our purpose is to...",<p><strong>$14.00 PER HOUR</strong></p>\n<p><s...,Service Technician (Will Train),Outside Service Technician,True,False,"[{'EndMonth': 3.0, 'EndYear': 2020.0, 'JobTitl...","[{'Degree': 'Certificate', 'Description': None...",...,K8yQlic+/UiXxBMpOnAoLQ==,Decline,2.0,declined,False,0,Security Contractor++Patrolman++Security Offic...,always be alert and ready to go in the case of...,Certificate++High School Diploma,Report writing++Training++Security++Weapons++D...


In [19]:
data.columns

Index(['OpportunityId', 'ApplicationId', 'ExternalBriefDescription',
       'ExternalDescription', 'Title', 'JobCategoryName', 'IsRejected',
       'IsCandidateInternal', 'WorkExperiences', 'Educations',
       'LicenseAndCertifications', 'Skills', 'Motivations', 'Behaviors',
       'StepId', 'StepName', 'Tag', 'StepGroup', 'pass_first_step',
       'Step_Category', 'Applicant_Job_Titles',
       'Applicant_Job_Responsibilities', 'Applicant_Education',
       'Applicant_Reported_Skills'],
      dtype='object')

In [20]:
cols_to_drop = ['WorkExperiences', 'Educations', 'Skills', "JobCategoryName",
                'IsCandidateInternal', 'WorkExperiences', 'Educations',
                'LicenseAndCertifications', 'Skills', 'Motivations', "IsRejected",'Behaviors',
                'StepId', 'StepName', 'Tag', 'StepGroup',]
data.drop(columns = cols_to_drop, inplace=True)

In [21]:
data

,OpportunityId,ApplicationId,ExternalBriefDescription,ExternalDescription,Title,pass_first_step,Step_Category,Applicant_Job_Titles,Applicant_Job_Responsibilities,Applicant_Education,Applicant_Reported_Skills
66159,x3PXnFA1GkCs0Cdz7q83zA==,gICe00fHwUKHC5ROv8WM4g==,"At Orkin, our purpose is to help protect the w...","<p><strong>At Orkin, our purpose is to help pr...",Termite Technician,True,1,General Manager++Assistant Manager++General Ma...,30907\nInterviewed job candidates and made sta...,High School Diploma++BPA,Sales++Cash++Customer service++Marketing++Rece...
66160,uEY0wW08R0WpiBkds/p4fg==,aaQVNXk5OEqPL7kZJJW6iw==,"At Orkin, our purpose is to help protect the w...","<p><strong>At Orkin, our purpose is to help pr...",Termite Technician,False,0,Package handler,It waa a great job just got home to late in th...,,
66161,gbzxt0dALU6x83ACFTSGEA==,mggx+MQSvUG6prEI0WkoAA==,"At Orkin, our purpose is to help protect the w...","<p><strong>At Orkin, our purpose is to help pr...",Termite Technician,False,0,Logistics Supervisor++Logistics Warehouse Supe...,* Supervised operations in multi-shift food gr...,Bachelor of Science++Master of Arts,Metrics++Inventory++Training++Operations++Hr++...
66162,Etqs/xDAX0SXrfmQA6+KBQ==,ewtbense/kyY62nuXSljHw==,"At Orkin, our purpose is to help protect the w...","<p><strong>At Orkin, our purpose is to help pr...",Termite Technician,False,0,Service tech,Diagonose and repair vehicles. Suspension work...,,
66163,lOZhR1k1Kk+OVz6n8r9vwA==,OuoS29IERk+mJTjddJNYSQ==,"At Orkin, our purpose is to help protect the w...","<p><strong>At Orkin, our purpose is to help pr...",Termite Technician,False,0,"security supervisor ++Sharp Shooter, Navigator...","Patrolling, protecting people, property, infor...",G.E.D,Cpr++Cpr certified++Documenting++Radio communi...
...,...,...,...,...,...,...,...,...,...,...,...
88207,5hdHoZVOPUWJqeggPv5T1Q==,Yi4u0MpbW0CyhqdpFwQvmA==,"At Orkin, our purpose is to help protect the w...","<p><strong>At Orkin, our purpose is to help pr...",Pest Control Specialist,False,0,Delivery Driver,Delivered and sold furniture,,
88208,ULIT1Ap/WEeVR6sinqxo9w==,ENwvF9XYsU2yuoI0paNsfg==,"At Orkin, our purpose is to help protect the w...","<p><strong>At Orkin, our purpose is to help pr...",Pest Control Specialist,False,0,Cook,"Cooking food, keeping hot box stocked, cleanin...",Diploma,Driving
88209,oAZjGHneVkKWKsiRaGh8Ag==,+9UVMbL9kUa6VstH0Bzgnw==,"$16.00 Per Hour\n\nAt Orkin, our purpose is to...",<p><strong>$16.00 Per Hour</strong></p>\n<p><s...,Service Technician (Will Train),True,0,Auditor,,Diploma,Confined space training
88210,oAZjGHneVkKWKsiRaGh8Ag==,7RyhIseedUGAVV4lfsKGow==,"$16.00 Per Hour\n\nAt Orkin, our purpose is to...",<p><strong>$16.00 Per Hour</strong></p>\n<p><s...,Service Technician (Will Train),True,0,Machine Operator++Thrower++Customer Service Re...,Simple machine operations++On the back of the ...,Diploma,Customer service++Customer service representat...


In [22]:
data.to_parquet("../../Data/split_4_cleaned.parquet")

### Here, we cleaned the application data by removing columns we won't need for the model.
- People interview for roles they're qualified for, and that's typically based on their skills, previous experience, and education.
- Therefore, we've taken applicant data such as job titles, experiences, skills, and education.
- We have held onto opportunity columns that provide the requirements for the role (ExternalDescription and ExternalBriefDescription) as well as job title (so we can infer some skills necessary for the role, if we choose to)
- We also have identifying information (OpportunityId and ApplicationId) as unique identifiers
- To create a supervised learning algorithm, we also have a variable which we can use to see whether or not the applicant made it to the next steps, such as phone screenings and interviews.